# Подготовка изображений и тестирование модели TF Lite

In [1]:
!pip install keras_image_helper

In [2]:
from keras_image_helper import create_preprocessor

In [3]:
# Создаем функцию-препроцессинга для последующего преобразования в массив Nympy

preprocessor = create_preprocessor('xception', target_size=(299, 299))

http://bit.ly/mlbookcamp-pants:

![alt text](http://bit.ly/mlbookcamp-pants)


In [4]:
image_url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(image_url)

In [5]:
X.shape

(1, 299, 299, 3)

- 1 - количество изображений
- 299 - высота
- 299 - ширина
- 3 - количество цветовых каналов: красный, зеленый и синий

In [6]:
import tensorflow.lite as tflite
# import tflite_runtime.interpreter as tflite

In [7]:
interpreter = tflite.Interpreter(model_path='clothing-model-v4.tflite') # создает интерпретатор
interpreter.allocate_tensors() # инициализирует интерпретатор с помощью модели

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Получаем входные данные (куда пойдет Х) и выходные данные (откуда получим прогнозы)

In [8]:
input_details = interpreter.get_input_details()
input_index = input_details[0]['index']

output_details = interpreter.get_output_details()
output_index = output_details[0]['index']

Делаем предсказания:

In [9]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()

preds = interpreter.get_tensor(output_index)

Переменная preds содержит вероятности принадлежности к каждому классу:

In [10]:
preds

array([[-0.1974976, -3.2490466, -0.6165978,  0.2244904,  9.353989 ,
        -0.6033057, -3.8435664,  4.8904605, -0.3794309, -3.1348174]],
      dtype=float32)

Конвертируем предсказания в читаемый формат:

In [11]:
labels = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [12]:
results = dict(zip(labels, preds[0]))
results

{'dress': -0.1974976,
 'hat': -3.2490466,
 'longsleeve': -0.6165978,
 'outwear': 0.2244904,
 'pants': 9.353989,
 'shirt': -0.6033057,
 'shoes': -3.8435664,
 'shorts': 4.8904605,
 'skirt': -0.3794309,
 't-shirt': -3.1348174}

Метка pants имеет наибольшую вероятность, что означает, что модель считает, что на изображении изображены брюки.